In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import trapz, cumtrapz
from posterior_inference_shared import Distribution_f
from posterior_inference_point import PointSourcePosterior, Distribution_N_gamma
from posterior_inference_diffuse import DiffusePosterior
import os
from scipy.integrate import cumtrapz

colors = [c["color"] for c in plt.rcParams['axes.prop_cycle']]

In [2]:
m_pbh = 0.5
n_pbh = 1
n_u = 19
merger_rate_prior = 'LF'
lambda_prior = 'LF'

# Diffuse limits

Under the null hypothesis, the likelihood for the fluxes in each bin follows a $\chi^2$ distribution, so this should be our test statistic. The limit on $\langle \sigma v \rangle$ is set by fixing $P(\chi^2 \geq \chi^2_{\mathrm{crit}}) = \alpha$, where $\alpha = 0.95$. For 26 bins, this translates to $\chi^2_{\mathrm{crit}} = 38.9$.

Since the likelihood is
$$
p(\mathcal{D} | N_{\mathrm{PBH}}, \langle \sigma v \rangle) = \int df\ p(f | N_{\mathrm{PBH}}) \prod_{i=1}^{26} \frac{1}{\sqrt{2\pi} \sigma_i} \exp\left[ -\frac{1}{2} \left( \frac{\phi_{\mathrm{PBH}}^{\mathrm{ex}}(E_i)}{\sigma_i} \right)^2 \right],
$$
we need to compute $p(\chi^2 | N_{\mathrm{PBH}}, \langle \sigma v \rangle)$ using Monte Carlo.

In [4]:
p_f = Distribution_f(m_pbh, n_pbh, merger_rate_prior)
f = p_f.sample(10)

In [6]:
from constants import e_egb, err_high_egb
from diffuse_constraints import phi_ex

In [ ]:
def sample_xi2(m_dm, sv):
    pass
#     1 / np.sqrt(2 * np.pi * err_high_egb**2) * np.exp(-phi_ex(e_egb, m_dm, ))

# Point source limits

In [ ]:
Distribution_N_gamma(m_pbh, )

In [9]:
from constants import phi_egb

In [3]:
post_ps = PointSourcePosterior(m_pbh, n_pbh, n_u, merger_rate_prior, lambda_prior)

In [4]:
post_ps._p_n_gamma._load_p_gamma_table()

In [7]:
post_ps.p_gamma(3e-30, 100.)

array([2.24077629e-07])